# Build Chatbot
`11_chatbot.ipynb`
- https://python.langchain.com/docs/tutorials/chatbot/

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi!, I am bob.'),
    AIMessage(content='Hello bob. how can I help you.'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

res = llm.invoke(messages)

res.pretty_print()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builider
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node 끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [ ]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방 아이디 (바뀌면 다른 대화가 된다.)

graph.invoke({'messages': messages}, config=config)

In [ ]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}}  # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='say my name.')
]
graph.invoke({'messages': messages}, config=config)

## Langgraph + `PromptTemplate`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 해적처럼 말해야해. 대항해 시대 해적을 최대한 따라해 봐.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

content='너는 해적처럼 말해야해. 대항해 시대 해적을 최대한 따라해 봐.' additional_kwargs={} response_metadata={}
content='hi' additional_kwargs={} response_metadata={}


In [30]:
builder = StateGraph(state_schema=MessagesState)

def simple_node(state: MessagesState):
    # prompt 추가.
    
    # prompt = prompt_template.invoke(state)
    # res = llm.invoke(prompt)
    
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [31]:
config = {'configurable': {'thread_id': 'qwer1234'}}  
graph.invoke({'messages': [HumanMessage(content='여기 한국인데')]}, config)

{'messages': [HumanMessage(content='여기 한국인데', additional_kwargs={}, response_metadata={}, id='a516939d-89ed-46aa-96a7-2f523c7c9a6a'),
  AIMessage(content='아하, 젠장! 한국이란 말이군! 이 해적의 눈으로 보면, 이 땅은 아시아의 보물섬이로군! 바다를 가로지르며 금은보화와 비밀을 찾아 떠나는 모험이 기다리고 있네! 배를 띄우고, 항해를 시작하자! 어디로 향할지, 어떤 전설을 만들어갈지 기대가 크군! 준비됐나, 후후!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 38, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_04d3664870', 'id': 'chatcmpl-CDKoGdp9pODVoGuZJJoaqK2BDMSEh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7c2ca729-23a1-4f01-acb6-90b444ab83ac-0', usage_metadata={'input_tokens': 38, 'output_tokens': 103, 'total_tokens': 141, 'input_token_details'